In [220]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from datetime import datetime

In [43]:
# read in the csv data
gradebook = pd.read_csv('C:/Users/noahh/Dropbox/p3reu/data/gradebooks/F17_CS112.csv')
pytania_s = pd.read_csv('C:/Users/noahh/Dropbox/p3reu/data/pytania/F17_CS112s_results.csv', encoding = "latin1")
zybook = pd.read_csv('C:/Users/noahh/Dropbox/p3reu/data/zyBook_112/GMUCS112Fall2017.csv')

In [49]:
# user ids for students in Snyder's section
s_ids = pytania_s['username']
# NOTE: there are 5 users in here with no entry in the gradebook! (maybe dropped the class early?)
users = set(s_ids.unique()) 
len(users)

214

In [70]:
df = gradebook.set_index('Username') # use unique ids as the index
df.sort_index(inplace=True) # sort by user id

In [46]:
gradebook_dict = df.to_dict(orient='index')

In [69]:
gradebook_s_dict = {} # grades for all students who were in Snyder's section of CS 112, Fall 2017
for user in gradebook_dict:
    if user in users:
        gradebook_s_dict[user] = gradebook_dict[user]

In [68]:
back_to_df = pd.DataFrame.from_dict(gradebook_s_dict, orient='index')
back_to_df.head()

,LEC,LAB,LETTER,RAW,HC,FF,T1,T2,T3,AVG_LABS_2drop,...,L4T,L5E,L6Q,L7E,L8Q,L9E,L10E,L11E,L12T,L13E
8,3,228,A+,99.1,0,0,90.5,95.5,89.0,100.0,...,100.0,100.0,80.0,100,75,100,100,100,100,100
35,3,211,F,34.9,0,0,61.0,34.0,0.0,55.4,...,100.0,0.0,30.0,64,60,0,0,0,0,0
56,3,228,A,93.0,0,0,93.0,82.0,82.0,99.6,...,100.0,100.0,75.0,96,95,100,100,80,100,100
61,3,224,A,94.6,0,0,88.0,91.0,91.5,99.1,...,100.0,100.0,75.0,96,95,100,100,0,0,100
70,3,213,A+,100.0,0,0,92.0,92.0,92.0,100.0,...,100.0,100.0,90.0,0,100,100,100,100,100,100


In [185]:
grades = back_to_df
X = grades[['T1','L1E','L2E','L3E','L4T','L5E','L6Q','P1','P2','P3']] # every lab and project up to test 1
y = grades['RAW']
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [186]:
classifier = LinearRegression()
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
math.sqrt(mean_squared_error(y_test, prediction))

10.804977421743157

In [200]:
def letter_grade(raw_score, precise=False):
    letter = ''
    if raw_score >= 98:
        letter = 'A+'
    elif raw_score >= 92:
        letter = 'A'
    elif raw_score >= 90:
        letter = 'A-'
    elif raw_score >= 88:
        letter = 'B+'
    elif raw_score >= 82:
        letter = 'B'
    elif raw_score >= 80:
        letter = 'B-'
    elif raw_score >= 78:
        letter = 'C+'
    elif raw_score >= 72:
        letter = 'C'
    elif raw_score >= 70:
        letter = 'C-'
    elif raw_score >= 60:
        letter = 'D'
    else:
        letter = 'F'
    
    if precise:
        return letter
    else:
        if letter in ['A+','A','A-']:
            return 'A'
        elif letter in ['B+','B','B-']:
            return 'B'
        elif letter in ['C+','C','C-']:
            return 'C'
        else:
            return letter

In [201]:
letter_predictions = []
for score in prediction:
    letter = letter_grade(score)
    letter_predictions.append(letter)

actual_letters = []
for score in y_test:
    letter = letter_grade(score)
    actual_letters.append(letter)
    
accuracy_score(actual_letters, letter_predictions)

0.6037735849056604

In [229]:
# converting timestamp strings into datetime objects that can be compared more easily
timestamps = pytania_s['timestamp']
example = list(timestamps)[0]
print(example)
datetime.strptime(example,'%Y-%m-%d %H:%M:%S%z')

2018-08-22 14:13:04+00:00


datetime.datetime(2018, 8, 22, 14, 13, 4, tzinfo=datetime.timezone.utc)